In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
#from localcider.sequenceParameters import SequenceParameters
import scipy
from scipy.spatial.distance import cdist
import math

In [23]:
def split_sphere(center ,R = 1, horizontal_split = 36, vertical_split = 36):
    theta = np.linspace(0,360,horizontal_split+1)
    c = np.linspace(-1, 1, vertical_split+1)
    s = 1 - c**2
    x = R * np.outer(s, np.cos(theta))
    x += center[0]
    y = R * np.outer(s, np.sin(theta))
    y += center[1]
    z = R * np.outer(c, np.ones(horizontal_split+1))
    z += center[2]
    return x, y, z

In [2]:
from function import salt_bridge
pdb_path = './data/pdbs'
test = salt_bridge(pdb_path, 'AF-C0H3V8-F1.pdb')

NameError: name 'np' is not defined

In [4]:
def distance (array1, array2, cutoff):
    distance = cdist(array1[:,2:], array2[:,2:], metric='euclidean') #calculate distance
    distance = np.concatenate((np.array([array2[:,0:1]]), distance), axis=0) #add atom number from  array2
    distance = np.concatenate((np.insert(np.array([array1[:,0:1]]), 0, None).reshape(-1,1), distance), axis=1) #add atom number from array1
    distance[2:, 2:][distance[2:, 2:] > cutoff] = np.nan #set distance > cutoff to nan
    rows_with_nan = np.insert(np.array([np.all(np.isnan(distance[2:, 2:]), axis=1)]),0, None) #find rows with all nan values
    cols_with_nan = np.insert(np.array([np.all(np.isnan(distance[2:, 2:]), axis=0)]),0, None) #find columns with all nan values
    distance = distance[~rows_with_nan, :] #delete rows with all nan values
    distance = distance[:, ~cols_with_nan] #delete columns with all nan values
    return distance



In [10]:
pdb_path = './data/pdbs'
with open(os.path.join(pdb_path, 'AF-C0H3V8-F1.pdb')) as f:
    AA_array = np.empty((0, 4))
    VdW_radii = {'C': 3.1, 'N': 2.95, 'O': 2.96} # Van der Waals radii in Angstrom enlarged by watermolecule radius 1.4 A (https://academic.oup.com/nar/article/49/W1/W559/6279848#267025710)
    C_C = 6.2
    C_N = 6.05
    C_O = 6.06
    N_N = 5.9
    N_O = 5.91
    O_O = 5.92
    C_array = np.empty((0,4))
    N_array = np.empty((0,4))
    O_array = np.empty((0,4))
    for line in f:
        line = line.strip()
        if line.startswith('ATOM'):


#select all Atoms from LEU, VAL, ILE that are within 1.4 A
    
   


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [58]:
test = np.array([1])
print(test.ndim)


1
